In [44]:
'''Import statements.'''

import pandas as pd
import altair as alt

In [45]:
'''Loading and cleaning data.'''

district_df = pd.read_csv("./data/ward_vdcmun_district_name_mapping.csv")
district_df.drop_duplicates(subset=['district_id', 'district_name'], inplace=True)
district_df = district_df[['district_id', 'district_name']]

def collapse_grade(ft_pct):
    pct = ft_pct * 100

    if pct >= 95:
        return False
    
    else:
        return True
    
def attachment(attach):
    if attach == 'Not attached':
        return False
    
    return True

df = pd.read_csv('./data/csv_building_structure.csv').dropna()

bool_col = [col for col in df.columns if 'has' in col]

for col in bool_col:
    df[col] = df[col].astype(bool)

df['damage_grade'] = df['damage_grade'].apply(lambda x: int(x.strip('Grade')))

df['collapse_grade'] = (df['height_ft_post_eq'] \
/ df['height_ft_pre_eq']).apply(lambda x: collapse_grade(x))

df['position'] = df['position'].apply(lambda x: attachment(x))

df = pd.merge(df, district_df, on='district_id')

df.head()

,building_id,district_id,vdcmun_id,ward_id,count_floors_pre_eq,count_floors_post_eq,age_building,plinth_area_sq_ft,height_ft_pre_eq,height_ft_post_eq,...,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,condition_post_eq,damage_grade,technical_solution_proposed,collapse_grade,district_name
0,120101000011,12,1207,120703,1,1,9,288,9,9,...,False,True,False,False,False,Damaged-Used in risk,3,Major repair,False,Okhaldhunga
1,120101000021,12,1207,120703,1,1,15,364,9,9,...,False,True,False,False,False,Damaged-Repaired and used,5,Reconstruction,False,Okhaldhunga
2,120101000031,12,1207,120703,1,1,20,384,9,9,...,False,False,False,False,False,Damaged-Repaired and used,2,Minor repair,False,Okhaldhunga
3,120101000041,12,1207,120703,1,1,20,312,9,9,...,False,False,False,False,False,Damaged-Repaired and used,2,Minor repair,False,Okhaldhunga
4,120101000051,12,1207,120703,1,1,30,308,9,9,...,False,False,False,False,False,Damaged-Repaired and used,1,Minor repair,False,Okhaldhunga


In [46]:
collapse_count = df.groupby(['district_name', 'collapse_grade']).count().reset_index()

dom = [True, False] 
rng = ['black', 'chocolate'] 

collapse_by_district = alt.Chart(collapse_count, 
title='Building Collapse By District').mark_bar(
    opacity=0.8
).encode(
    x=alt.X('collapse_grade:N', title=None).axis(),
    y=alt.Y('building_id', title='Building Count'),
    color=alt.Color('collapse_grade:N', title='Building collapsed?').scale(domain=dom, range=rng),
    column=alt.Column('district_name', title=None)
)

collapse_by_district.properties(title = alt.TitleParams(
    text = "District Dinstinctions: Differences in Damage Levels by District", 
    subtitle = ["Not all districts felt the same impact of the 2015 Ghorka earthquake, with Sindhupalchok suffering the most damage out of all districts."],
    fontSize = 28,
    color = '#3E454F', 
    subtitleFont = 'Barlow',
    subtitleFontSize = 16, 
    subtitleColor = '#3E454F'),
    width=45,
    height=450
)

alt.Chart(...)

In [94]:
building_foundation = df.groupby(['district_name', 
'foundation_type']).count().reset_index()

dom = ['Bamboo/Timber', 'Cement-Stone/Brick', 'Mud mortar-Stone/Brick', 
       'Other', 'RC'] 
rng = ['lightgreen', 'dimgray', 'sienna', 'dodgerblue', 'black'] 

foundation_by_district = alt.Chart(building_foundation, 
title='Land Condition By District').mark_bar(
    opacity=0.8
).encode(
    x=alt.X('foundation_type:N', title=None).axis(labels=False),
    y=alt.Y('building_id', title='Building Count'),
    color=alt.Color('foundation_type:N', title='Foundation Type').scale(domain=dom, range=rng),
    column=alt.Column('district_name', title=None)
)

foundation_by_district.properties(title = alt.TitleParams(
    text = "Laying the Foundations: Explaining the Difference in Damage by Districts", 
    subtitle = ["Most buildings are built on Mud mortar-stone/Brick, but Bamboo/Timber are prevalent in Makwanpur & Sindhuli which could explain better building survival rates."],
    fontSize = 28,
    color = '#3E454F', 
    subtitleFont = 'Barlow',
    subtitleFontSize = 16, 
    subtitleColor = '#3E454F'),
    width=80,
    height=450
)

alt.Chart(...)

In [97]:
mak_sindhu_collapse = collapse_count[collapse_count['district_name'].apply(lambda x: x in ['Makwanpur', 'Sindhupalchok'])]
"Despite being equidistant from the epicenter, Sindhupalchok & Makwanpur had very different experiences which is partly attributable to the difference in foundation materials."

dom = [True, False] 
rng = ['firebrick', 'midnightblue'] 

collapse_mak_sindhu = alt.Chart(mak_sindhu_collapse).mark_bar(
    opacity=0.8
).encode(
    x=alt.X('collapse_grade:N', title=None).axis(labels=False),
    y=alt.Y('building_id', title='Building Count'),
    color=alt.Color('collapse_grade:N', title='Building collapsed?').scale(domain=dom, range=rng),
    column=alt.Column('district_name', title=None)
).properties(
    width=100,
    height=450
)

mak_sindhu_foundation = building_foundation[building_foundation['district_name'].apply(lambda x: x in ['Makwanpur', 'Sindhupalchok'])]

dom = ['Bamboo/Timber', 'Cement-Stone/Brick', 'Mud mortar-Stone/Brick', 
       'Other', 'RC'] 
rng = ['lightgreen', 'dimgray', 'sienna', 'dodgerblue', 'black'] 

foundation_mak_sindhu = alt.Chart(mak_sindhu_foundation).mark_bar(
    opacity=0.8
).encode(
    x=alt.X('foundation_type', title=None).axis(labels=False),
    y=alt.Y('building_id', title='Building Count'),
    color=alt.Color('foundation_type', title='Foundation Type').scale(domain=dom, range=rng),
    column=alt.Column('district_name', title=None)
).properties(
    width=250,
    height=450
)

mak_sindhu = alt.hconcat(
    collapse_mak_sindhu, foundation_mak_sindhu
).resolve_scale(
    color='independent'
).configure_view(
    stroke=None
)

mak_sindhu.properties(title = alt.TitleParams(
    text = "A Tale of Two Districts: Comparing Makwanpur & Sindhupalchok", 
    subtitle = ["Despite being equidistant from the epicenter, Sindhupalchok & Makwanpur had very different experiences which is partly attributable to the difference in foundation materials."],
    fontSize = 28,
    color = '#3E454F', 
    subtitleFont = 'Barlow',
    subtitleFontSize = 16, 
    subtitleColor = '#3E454F'),
)

alt.HConcatChart(...)

In [48]:
foundation = df.groupby(['collapse_grade', 'foundation_type']).count().reset_index()

dom = [True, False] 
rng = ['black', 'sienna'] 

foundation_collapse = alt.Chart(foundation, 
title='Land Condition By Collapse Grade').mark_bar(
    opacity=0.8
).encode(
    x=alt.X('foundation_type:N', title='Foundation Type').axis(),
    y=alt.Y('building_id', title='Building Count'),
    color=alt.Color('collapse_grade', title='Building Collapsed?').scale(domain=dom, range=rng))

foundation_collapse.properties(title = alt.TitleParams(
    text = "Calling All Reinforcement: General Foundation Performance", 
    subtitle = ["Reinforced Concrete (RC) foundations are the most likely to hold up while mud buildings are likely to crumble."],
    fontSize = 28,
    color = '#3E454F', 
    subtitleFont = 'Barlow',
    subtitleFontSize = 16, 
    subtitleColor = '#3E454F'),
    width=600,
    height=450
)

alt.Chart(...)

In [52]:
quantized_df = df.copy()

quantized_df['age_building_q'] = pd.qcut(df['age_building'], 20, labels=False)
quantized_df['plinth_area_sq_ft_q'] = pd.qcut(df['plinth_area_sq_ft'], 20, labels=False)

quantized_df

,building_id,district_id,vdcmun_id,ward_id,count_floors_pre_eq,count_floors_post_eq,age_building,plinth_area_sq_ft,height_ft_pre_eq,height_ft_post_eq,...,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,condition_post_eq,damage_grade,technical_solution_proposed,collapse_grade,district_name,age_building_q,plinth_area_sq_ft_q
0,120101000011,12,1207,120703,1,1,9,288,9,9,...,False,False,False,Damaged-Used in risk,3,Major repair,False,Okhaldhunga,4,5
1,120101000021,12,1207,120703,1,1,15,364,9,9,...,False,False,False,Damaged-Repaired and used,5,Reconstruction,False,Okhaldhunga,8,10
2,120101000031,12,1207,120703,1,1,20,384,9,9,...,False,False,False,Damaged-Repaired and used,2,Minor repair,False,Okhaldhunga,11,11
3,120101000041,12,1207,120703,1,1,20,312,9,9,...,False,False,False,Damaged-Repaired and used,2,Minor repair,False,Okhaldhunga,11,7
4,120101000051,12,1207,120703,1,1,30,308,9,9,...,False,False,False,Damaged-Repaired and used,1,Minor repair,False,Okhaldhunga,15,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
762089,366709001241,36,3603,360302,2,0,60,165,18,0,...,False,False,False,Damaged-Rubble clear,5,Reconstruction,True,Gorkha,19,0
762090,366709001251,36,3603,360302,2,0,35,342,18,0,...,False,False,False,Damaged-Rubble clear,5,Reconstruction,True,Gorkha,16,9
762091,366709001261,36,3603,360302,2,0,35,342,18,0,...,False,False,False,Damaged-Rubble clear,5,Reconstruction,True,Gorkha,16,9
762092,366709001271,36,3603,360302,2,0,19,306,18,0,...,False,False,False,Damaged-Rubble clear,5,Reconstruction,True,Gorkha,11,7


In [56]:
age = quantized_df[['building_id', 'collapse_grade', 'age_building_q']].groupby(['collapse_grade', 
'age_building_q']).count().reset_index()

intact_age = age[age['collapse_grade'] == False].copy()
intact_age['building_id'] = intact_age['building_id'] / sum(intact_age['building_id'])

collapse_age = age[age['collapse_grade'] == True].copy()
collapse_age['building_id'] = collapse_age['building_id'] / sum(collapse_age['building_id'])

age_norm = pd.concat([intact_age, collapse_age])

age_norm

dom = [True, False] 
rng = ['firebrick', 'midnightblue']


age_C = alt.Chart(age_norm).mark_bar(
    opacity=0.7,
    ).encode(
    x=alt.X('age_building_q:N', title='Building Age Quantile'),
    y=alt.Y('building_id:Q', title=' Normalized Concentration').stack(None),
    color=alt.Color('collapse_grade', title='Building Collapsed?').scale(domain=dom, 
    range=rng)
).configure_scale(
    bandPaddingInner=0
)

age_C.properties(title = alt.TitleParams(
    text = "With Age Comes... Instability", 
    subtitle = ["Older buildings are incredibly vulnerable to seismic stability."],
    fontSize = 28,
    color = '#3E454F', 
    subtitleFont = 'Barlow',
    subtitleFontSize = 16, 
    subtitleColor = '#3E454F'),
    width=600,
    height=450
)

alt.Chart(...)

In [57]:
plinth = quantized_df[['building_id', 'collapse_grade', 'plinth_area_sq_ft_q']].groupby(['collapse_grade', 
'plinth_area_sq_ft_q']).count().reset_index()

intact_plinth = plinth[plinth['collapse_grade'] == False].copy()
intact_plinth['building_id'] = intact_plinth['building_id'] / sum(intact_plinth['building_id'])

collapse_plinth = plinth[plinth['collapse_grade'] == True].copy()
collapse_plinth['building_id'] = collapse_plinth['building_id'] / sum(collapse_plinth['building_id'])

plinth_norm = pd.concat([intact_plinth, collapse_plinth])

dom = [True, False] 
rng = ['firebrick', 'midnightblue'] 


plinth_C = alt.Chart(plinth_norm).mark_bar(
    opacity=0.7,
    ).encode(
    x=alt.X('plinth_area_sq_ft_q:N', title='Building Age Quantile'),
    y=alt.Y('building_id:Q', title='Normalized Concentration').stack(None),
    color=alt.Color('collapse_grade', title='Building Collapsed?').scale(domain=dom, 
    range=rng)
).configure_scale(
    bandPaddingInner=0
)

plinth_C.properties(title = alt.TitleParams(
    text = "Back to the Basics", 
    subtitle = ["Opting for a larger plinth (foundational area) of a building can increase seismic stability."],
    fontSize = 28,
    color = '#3E454F', 
    subtitleFont = 'Barlow',
    subtitleFontSize = 16, 
    subtitleColor = '#3E454F'),
    width=600,
    height=450
)

alt.Chart(...)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=94503a49-0c4d-4334-846d-2f5f3164b959' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>